# Barabasi-Albert model

In [1]:
import networkx as nx
import numpy as np
import random as rd
import matplotlib.pyplot as plt

In [ ]:
COLOR = '#40a6d1'

## Parameters

The algorithm hase 3 input parameters: 
* m_0: number of initial nodes ( > 0 )
* N: number of final nodes
* m: number of links for each new added node (should be m < m_0)

Insert the parameters of the algorithm (need to add the checks!!!!!!!!!)

In [3]:
# Insert Parameters
m_0 = int(input("Insert he number of initial nodes:\n"))
N = int(input("\nInsert number of final nodes:\n"))
m = int(input("\nInsert in the value of m parameter:\n"))

Insert he number of initial nodes:
50

Insert number of final nodes:
1000

Insert in the value of m parameter:
5


Let's now create the initial graph: we build a complete graph with m_0 nodes

In [4]:
G = nx.complete_graph(m_0) # nodes in the range from 0 to m_0

In [5]:
len(G.nodes())

50

In [6]:
# list the nodes
G.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49))

We now need to add incrementally nodes until N (we'll add nodes by increasing the number intefying each node). 
We use a loop to do that. After adding a node we need to add m links from it to the existing nodes, we'll use another (nested) loop to do that.

## Auxiliary functions
Before implementing the actual code of the algorithm, lets define some helper functions for creating m links between the newly added node and the existing nodes according to the prefential attachment mechanism

The first function we'll use defines the weighted probabilities of each already existing node.

In [12]:
# returns the list of the weighted probability of each existing node
def build_probs(G):
    nodes_probs = [] # our list
                            
    # compute the probability for each node
    for node in G.nodes():
        node_deg = G.degree(node)
        
        # probability of the node
        node_prob = node_deg / (2 * len(G.edges)) # N.B.: 2 * len(G.edges) returns the sum of the degrees of all the nodes, since each edge contributes to the degree of 2 nodes
        
        # insert into the list
        nodes_probs.append(node_prob)
    
    return nodes_probs

The 2nd auxiliary function adds m links between a node and the existing nodes, according to the preferential attachment.
We use the random.choice() function of the numpy module, which takes as input the list of nodes among which to select one and the probability associated to each node.

In [13]:

def add_links(G, added_node, nodes_probs):
    selected_nodes = [] # list of the selected nodes
    
    # before inserting the links we build the weighted probabilities of each node
    nodes_probs = build_probs(G)
    
    # add m links
    for added_links in range(m):
        # we use the choice function of the numpy.random module
        selected_node = np.random.choice(G.nodes(),p=nodes_probs) # selects a node according to preferential attachment
        
        # check whether the node was already selected (to avoid multiple links)
        while selected_node in selected_nodes:
            selected_node = np.random.choice(G.nodes(),p=nodes_probs) # select another node
        
        # once we selected the node we should create the link among the added_node given as a parameter
        # and the selected_node
        G.add_edge(added_node, selected_node)
        

## Algorithm

In [17]:
nodes_to_add = N - m_0 # number of nodes to add until reaching N total nodes
inserted_nodes = 0 # counter of the inserted nodes

for n_i in range(nodes_to_add):
     
    # insert new_node in the graph
    print("----------> Step {} <----------".format(inserted_nodes + 1))
    G.add_node(m_0 + inserted_nodes) # we add the node by specifying its numeric value (starting from 50)
    added_node = m_0 + inserted_nodes # identifier of the added node
    inserted_nodes += 1
    
    print("Node added: {}".format(added_node))
    # add the newly inserted node into nodes_probs because of the random.choice function to work
    
    # --------------------------------------------------------------

    # after adding the node, we need to create m links with the existing nodes 
    # according to the preferential attachment mechanism
    
    # function to add m links to the newly inserted node
    add_links(G, added_node, nodes_probs)
    print('aaa')

----------> Step 1 <----------
Node added: 50
aaa
----------> Step 2 <----------
Node added: 51
aaa
----------> Step 3 <----------
Node added: 52
aaa
----------> Step 4 <----------
Node added: 53
aaa
----------> Step 5 <----------
Node added: 54
aaa
----------> Step 6 <----------
Node added: 55
aaa
----------> Step 7 <----------
Node added: 56
aaa
----------> Step 8 <----------
Node added: 57
aaa
----------> Step 9 <----------
Node added: 58
aaa
----------> Step 10 <----------
Node added: 59
aaa
----------> Step 11 <----------
Node added: 60
aaa
----------> Step 12 <----------
Node added: 61
aaa
----------> Step 13 <----------
Node added: 62
aaa
----------> Step 14 <----------
Node added: 63
aaa
----------> Step 15 <----------
Node added: 64
aaa
----------> Step 16 <----------
Node added: 65
aaa
----------> Step 17 <----------
Node added: 66
aaa
----------> Step 18 <----------
Node added: 67
aaa
----------> Step 19 <----------
Node added: 68
aaa
----------> Step 20 <----------
Node add

aaa
----------> Step 163 <----------
Node added: 212
aaa
----------> Step 164 <----------
Node added: 213
aaa
----------> Step 165 <----------
Node added: 214
aaa
----------> Step 166 <----------
Node added: 215
aaa
----------> Step 167 <----------
Node added: 216
aaa
----------> Step 168 <----------
Node added: 217
aaa
----------> Step 169 <----------
Node added: 218
aaa
----------> Step 170 <----------
Node added: 219
aaa
----------> Step 171 <----------
Node added: 220
aaa
----------> Step 172 <----------
Node added: 221
aaa
----------> Step 173 <----------
Node added: 222
aaa
----------> Step 174 <----------
Node added: 223
aaa
----------> Step 175 <----------
Node added: 224
aaa
----------> Step 176 <----------
Node added: 225
aaa
----------> Step 177 <----------
Node added: 226
aaa
----------> Step 178 <----------
Node added: 227
aaa
----------> Step 179 <----------
Node added: 228
aaa
----------> Step 180 <----------
Node added: 229
aaa
----------> Step 181 <----------
Node adde

aaa
----------> Step 323 <----------
Node added: 372
aaa
----------> Step 324 <----------
Node added: 373
aaa
----------> Step 325 <----------
Node added: 374
aaa
----------> Step 326 <----------
Node added: 375
aaa
----------> Step 327 <----------
Node added: 376
aaa
----------> Step 328 <----------
Node added: 377
aaa
----------> Step 329 <----------
Node added: 378
aaa
----------> Step 330 <----------
Node added: 379
aaa
----------> Step 331 <----------
Node added: 380
aaa
----------> Step 332 <----------
Node added: 381
aaa
----------> Step 333 <----------
Node added: 382
aaa
----------> Step 334 <----------
Node added: 383
aaa
----------> Step 335 <----------
Node added: 384
aaa
----------> Step 336 <----------
Node added: 385
aaa
----------> Step 337 <----------
Node added: 386
aaa
----------> Step 338 <----------
Node added: 387
aaa
----------> Step 339 <----------
Node added: 388
aaa
----------> Step 340 <----------
Node added: 389
aaa
----------> Step 341 <----------
Node adde

aaa
----------> Step 479 <----------
Node added: 528
aaa
----------> Step 480 <----------
Node added: 529
aaa
----------> Step 481 <----------
Node added: 530
aaa
----------> Step 482 <----------
Node added: 531
aaa
----------> Step 483 <----------
Node added: 532
aaa
----------> Step 484 <----------
Node added: 533
aaa
----------> Step 485 <----------
Node added: 534
aaa
----------> Step 486 <----------
Node added: 535
aaa
----------> Step 487 <----------
Node added: 536
aaa
----------> Step 488 <----------
Node added: 537
aaa
----------> Step 489 <----------
Node added: 538
aaa
----------> Step 490 <----------
Node added: 539
aaa
----------> Step 491 <----------
Node added: 540
aaa
----------> Step 492 <----------
Node added: 541
aaa
----------> Step 493 <----------
Node added: 542
aaa
----------> Step 494 <----------
Node added: 543
aaa
----------> Step 495 <----------
Node added: 544
aaa
----------> Step 496 <----------
Node added: 545
aaa
----------> Step 497 <----------
Node adde

aaa
----------> Step 635 <----------
Node added: 684
aaa
----------> Step 636 <----------
Node added: 685
aaa
----------> Step 637 <----------
Node added: 686
aaa
----------> Step 638 <----------
Node added: 687
aaa
----------> Step 639 <----------
Node added: 688
aaa
----------> Step 640 <----------
Node added: 689
aaa
----------> Step 641 <----------
Node added: 690
aaa
----------> Step 642 <----------
Node added: 691
aaa
----------> Step 643 <----------
Node added: 692
aaa
----------> Step 644 <----------
Node added: 693
aaa
----------> Step 645 <----------
Node added: 694
aaa
----------> Step 646 <----------
Node added: 695
aaa
----------> Step 647 <----------
Node added: 696
aaa
----------> Step 648 <----------
Node added: 697
aaa
----------> Step 649 <----------
Node added: 698
aaa
----------> Step 650 <----------
Node added: 699
aaa
----------> Step 651 <----------
Node added: 700
aaa
----------> Step 652 <----------
Node added: 701
aaa
----------> Step 653 <----------
Node adde

aaa
----------> Step 791 <----------
Node added: 840
aaa
----------> Step 792 <----------
Node added: 841
aaa
----------> Step 793 <----------
Node added: 842
aaa
----------> Step 794 <----------
Node added: 843
aaa
----------> Step 795 <----------
Node added: 844
aaa
----------> Step 796 <----------
Node added: 845
aaa
----------> Step 797 <----------
Node added: 846
aaa
----------> Step 798 <----------
Node added: 847
aaa
----------> Step 799 <----------
Node added: 848
aaa
----------> Step 800 <----------
Node added: 849
aaa
----------> Step 801 <----------
Node added: 850
aaa
----------> Step 802 <----------
Node added: 851
aaa
----------> Step 803 <----------
Node added: 852
aaa
----------> Step 804 <----------
Node added: 853
aaa
----------> Step 805 <----------
Node added: 854
aaa
----------> Step 806 <----------
Node added: 855
aaa
----------> Step 807 <----------
Node added: 856
aaa
----------> Step 808 <----------
Node added: 857
aaa
----------> Step 809 <----------
Node adde

aaa
----------> Step 947 <----------
Node added: 996
aaa
----------> Step 948 <----------
Node added: 997
aaa
----------> Step 949 <----------
Node added: 998
aaa
----------> Step 950 <----------
Node added: 999
aaa


In [ ]:
print(range(m))
for i in range(m):
    print (i)

#### Let's now check whether we obtained the desired model

In [ ]:
len(G.nodes())

In [ ]:
G.nodes()